In [29]:
from astropy.table import Table, MaskedColumn
from astropy.time import Time
import astropy.io.ascii
import numpy as np

In [30]:
traw = Table.read('../data/asset1_transactions.csv', format='ascii.csv', converters={
                  'quantity': [astropy.io.ascii.convert_numpy(np.float64)],
                  'unit price': [astropy.io.ascii.convert_numpy(np.float64)],
                  'total price': [astropy.io.ascii.convert_numpy(np.float64)]})

In [31]:
traw['date'] = Time(traw['date'])
traw.rename_column('date', 'dateS')
traw.add_column(MaskedColumn(traw['dateS'], name='dateE', mask=True))
traw.add_column(np.arange(1,len(traw)+1), name='pId')

In [32]:
traw

dateS,asset,type,quantity,unit price,total price,dateE,pId
object,str2,str4,float64,float64,float64,object,int64
2019-01-01 00:00:00.000,A1,buy,5.0,2.0,10.0,--,1
2020-01-01 00:00:00.000,A1,buy,10.0,3.0,30.0,--,2
2021-01-01 00:00:00.000,A1,sell,1.0,5.0,5.0,--,3


In [33]:
traw.sort(keys=['dateS',])

In [34]:
packets = traw[traw['type'] == 'buy']
packets.remove_columns(['type','total price'])
packets.rename_columns(['unit price'],['priceS'])
packets.add_column(MaskedColumn(np.zeros(len(packets)), name='priceE', mask=True))

In [35]:
packets

dateS,asset,quantity,priceS,dateE,pId,priceE
object,str2,float64,float64,object,int64,float64
2019-01-01 00:00:00.000,A1,5.0,2.0,--,1,--
2020-01-01 00:00:00.000,A1,10.0,3.0,--,2,--


In [36]:
selltable = traw[traw['type'] == 'sell']

In [37]:
traw

dateS,asset,type,quantity,unit price,total price,dateE,pId
object,str2,str4,float64,float64,float64,object,int64
2019-01-01 00:00:00.000,A1,buy,5.0,2.0,10.0,--,1
2020-01-01 00:00:00.000,A1,buy,10.0,3.0,30.0,--,2
2021-01-01 00:00:00.000,A1,sell,1.0,5.0,5.0,--,3


In [38]:
selltable

dateS,asset,type,quantity,unit price,total price,dateE,pId
object,str2,str4,float64,float64,float64,object,int64
2021-01-01 00:00:00.000,A1,sell,1.0,5.0,5.0,--,3


In [39]:
npId = len(traw)+1
for sale in selltable:
    i_consider = np.searchsorted(packets['dateS'], sale['dateS'], side='right')
    i_unclosed = np.flatnonzero(packets['dateE'].mask[:i_consider])
    saleUnits = sale['quantity']
    ii = 0
    while saleUnits > 0.:
        if len(i_unclosed) <= ii:
            raise ValueError("No match found for sale.")
        irec = i_unclosed[ii]
        packets['dateE'].mask[irec] = False        
        packets['dateE'][irec] = sale['dateS']
        packets['priceE'][irec] = sale['unit price']
        if saleUnits > packets['quantity'][irec]:
            saleUnits -= packets['quantity'][irec] # Still to match sale
        else:
            r = packets['quantity'][irec] - saleUnits
            packets.insert_row(irec+1, packets[irec])
            packets['quantity'][irec] = saleUnits
            irec += 1
            packets['quantity'][irec] = r
            packets['dateE'].mask[irec] = True
            packets['priceE'].mask[irec] = True
            packets['pId'][irec] = npId
            npId += 1
            saleUnits = 0.
        ii += 1

In [40]:
print(packets)

         dateS          asset quantity priceS          dateE          pId priceE
----------------------- ----- -------- ------ ----------------------- --- ------
2019-01-01 00:00:00.000    A1      1.0    2.0 2021-01-01 00:00:00.000   1    5.0
2019-01-01 00:00:00.000    A1      4.0    2.0                      --   4     --
2020-01-01 00:00:00.000    A1     10.0    3.0                      --   2     --


In [47]:
t1 = Time('2000-01-01')
t2 = Time('2000-04-15')

In [49]:
def getValue(t, valueT, k1, k2):
    if not np.all(valueT[k1][1:] - valueT[k1][:-1] >=0):
        raise ValueError("valueT is not monotonic increasing in dates")
    i = np.searchsorted(valueT[k1], t)
    if i < len(valueT):
        if valueT[k1][i] == t:
            return valueT[k2][i]
        if i==0:
            print("WARNING: Value data starts later than interested dates")
            return valueT[k2][0]
        vlast = valueT[k2][i-1]
        return vlast + (valueT[k2][i] - vlast) * (t - valueT[k1][i-1]) / (valueT[k1][i] - valueT[k1][i-1])
    else:
        print("WARNING: Value data ends earlier than interested dates")
        return valueT[k2][-1]

<Quantity 0.28767123>